In [1]:
!python --version

Python 3.11.6


In [2]:
!pip install openai


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# set openAI API key
import os
from openai import OpenAI
import pprint
import json
client = OpenAI()

In [4]:
def gptThink(chatConfig, toolMap=None):
    # Return new messages
    new_messages = []
    
    # Required parameters
    messages = chatConfig.get('messages')
    model = chatConfig.get('model')
    if messages is None or model is None:
        raise ValueError("The 'messages' and 'model' parameters are required.")
    
    # For demonstration, we'll just print the required parameters and additional ones.
    # for key, value in chatConfig.items():
    #     print(f"{key}: {pprint.pformat(value)}")

    response = client.chat.completions.create(**chatConfig)
    # print()
    # print("Response: " + pprint.pformat(response.model_dump()))
    if (response.choices[0].message.content):
        print("\nAssistant: \n" + response.choices[0].message.content)
   
    # append the messages but dump the None value in it.
    new_messages.append({k: v for k, v in response.choices[0].message.model_dump().items() if v is not None})

    # if have tool_calls, start recursive chat
    if (response.choices[0].message.tool_calls):
        for tool_call in response.choices[0].message.tool_calls:
            f = toolMap[tool_call.function.name]
            f_args = json.loads(tool_call.function.arguments)
            f_res = f(**f_args)
            message_obj = {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": tool_call.function.name,
                "content": f_res
            }
            print(f"\nfunction calling {tool_call.function.name}({f_args})\n" + pprint.pformat(message_obj))
            new_messages.append(message_obj)
        chatConfig['messages'] = messages + new_messages
        # print('chatConfig: ')
        # pprint.pprint(chatConfig)
        tc_response = gptThink(chatConfig, toolMap)
        new_messages = new_messages + tc_response
        # print(response.choices[0].message.content)
        
    return new_messages


In [7]:
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
]
def get_current_weather(location, unit="fahrenheit"):
    return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})

messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]

model="gpt-4"
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      }
    }
  }
]

In [8]:
gptThink({"messages":messages, "model":model, "tools": tools}, {"get_current_weather": get_current_weather})


function calling get_current_weather({'location': 'Boston, MA'})
{'content': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}',
 'name': 'get_current_weather',
 'role': 'tool',
 'tool_call_id': 'call_qT1nnlNdOiOip7ljK6ArIAr0'}

Assistant: 
The current temperature in Boston, MA is 10 degrees Celsius.


[{'role': 'assistant',
  'tool_calls': [{'id': 'call_qT1nnlNdOiOip7ljK6ArIAr0',
    'function': {'arguments': '{\n  "location": "Boston, MA"\n}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'tool_call_id': 'call_qT1nnlNdOiOip7ljK6ArIAr0',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}'},
 {'content': 'The current temperature in Boston, MA is 10 degrees Celsius.',
  'role': 'assistant'}]